In [1]:
import pandas as pd
import numpy as np
from lxml import etree
import bs4
from bs4 import BeautifulSoup as BS
import requests
import datetime


In [2]:
main_url = "https://gomafia.pro"
club_url = "/club/92"
history_url = "?tab=history"
YEAR = 2024

class Player():

    def __init__(self, data_row):

        self.nick = data_row[1].find('a').text
        self.link = main_url+data_row[1].find('a')['href']
        self.gg = int(data_row[3].text)
        self.elo = int(data_row[4].text)

        self.achievements = {}
        self.set_tournaments_achievements()

    def __repr__(self):
        return self.nick 

    def set_tournaments_achievements(self):
        player_hist_url = self.link + history_url
        year = YEAR
        i = 0
        while True:
            i += 1
            html_content = session.get(player_hist_url+'&page={}'.format(i)).content
            soup = BS(html_content, "html.parser")
            table = soup.find("table")
            if table is None:
                return
            for t in table.find('tbody').find_all('tr'):
                tour_link, tournament, nominations, d1, d2, place, gg = self.parse_tournament(t)
                d = datetime.datetime.strptime(d1, '%d.%m.%Y')
                t_year = d.year
                if t_year < year:
                    return
                current_achievments = {}
                if place <= 10:
                    current_achievments['place'] = place
                if gg > 0:
                    current_achievments['gg'] = gg
                if nominations:
                    current_achievments['nominations'] = nominations
                add_as_achiev = 'отбор' not in tournament.lower()

                if current_achievments and add_as_achiev:
                    current_achievments['date'] = d
                    self.achievements[tournament] = current_achievments

    @staticmethod
    def parse_tournament(tournament:bs4.Tag):
        '''Парсит все нужные данные о результатах игрока на конкретном турнире'''

        tour_link = tournament.find_all('td')[1].find('a')['href']
        tournament_name = tournament.find_all('td')[1].find('a').text
        nominations = []
        for i in tournament.find_all('td')[1].find_all('div'):
            if i.get('data-tooltip-html', None):
                nominations.append((i['data-tooltip-html']))
        d1 = tournament.find_all('td')[2].find_all('div', class_="undefined undefined")[0].text
        d2 = tournament.find_all('td')[2].find_all('div', class_="undefined undefined")[1].text
        place = int(tournament.find_all('td')[3].text)
        gg = int(tournament.find_all('td')[4].text)

        return tour_link, tournament_name, nominations, d1, d2, place, gg


club_players = {}
session = requests.Session()
j = 0
stop_flg = False
for i in range(1, 11):
    print(i)
    html_content = session.get(main_url+club_url+f'?page={i}').content
    soup = BS(html_content, "html.parser")
    table = soup.find("table")
    for row in table.find('tbody').find_all('tr'):
        j += 1
        data_row = row.find_all('td')
        if int(data_row[3].text) == 0:
            stop_flg = True
            break
        p = Player(data_row)
        club_players[j] = p
    if stop_flg:
        break

1
2
3


In [3]:
df_data = []
columns = ['nick', 'date', 'tour', 'place']
for i, pl in club_players.items():
    for t in pl.achievements:
        d = pl.achievements[t].get('date', None)
        place = pl.achievements[t].get('place', None)
        if place:
            df_data.append([pl.nick, d, t, place])

club_achievements = pd.DataFrame(df_data, columns=columns).sort_values('date')

In [4]:
# club_achievements.to_csv('club_achievements.csv', encoding='cp1251', sep=';')

In [5]:
club_achievements

,nick,date,tour,place
25,Воробушек,2024-01-06,CHALLENGER 2023,10
57,YungBull,2024-01-20,Mordovia Winter Team Championship 2024,4
122,crsms,2024-01-27,Четыре Чемпиона 2024,9
24,Воробушек,2024-01-27,MASTERS | NORTH 2023,4
33,Грозный,2024-02-10,Midnight Cup 2024,5
...,...,...,...,...
81,Мэд,2025-08-02,Squid Game Cup 2025,10
89,Endurance,2025-08-02,Squid Game Cup 2025,9
104,Ave 17,2025-08-09,Открытое первенство Черноземья (Тарантино Сup...,3
105,Сандра,2025-08-09,Кубок Северной Пальмиры 2025,8
